# ⚛️ H₂ VQE — Noisy Ansatz Comparison (Production API)

This notebook compares **ansatz families** for **H₂ VQE** under different
**noise channels**, using only the **packaged VQE API**.

We run **multi-seed noise statistics** and compare ansatzes by:

- Mean and standard deviation of the **energy error**:  ΔE = E_noisy − E_ref
- Mean and standard deviation of the **fidelity** vs a noiseless reference state
- Across a grid of **noise strengths**

Key entry point:

```python
from vqe.core import run_vqe_ansatz_comparison
```

## 1️⃣ Imports and configuration

In [1]:
import os
import sys
import numpy as np

# Allow local import when running from notebooks/
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../..")))

from vqe.core import run_vqe_ansatz_comparison

### Experiment defaults

- Molecule: **H₂**
- Optimizer: **Adam**
- Steps / stepsize: **50**, **0.2**
- Mapping: **Jordan–Wigner**
- Noise grid: **0.00 → 0.10**
- Seeds: **0 → 9**

$$\text{cost} \sim N_{\text{ansatz}} \times N_{\text{noise}} \times N_{\text{seeds}} \times N_{\text{steps}}$$

In [ ]:
molecule = "H2"
optimizer_name = "Adam"
mapping = "jordan_wigner"

steps = 50
stepsize = 0.2

noise_levels = np.arange(0.0, 0.11, 0.02)
seeds = np.arange(0, 10)

ansatzes = [
    "UCC-S",
    "UCC-D",
    "UCCSD",
    "Minimal",
    "RY-CZ",
    "TwoQubit-RY-CNOT",
    "StronglyEntanglingLayers",
]

print("Molecule:", molecule)
print("Optimizer:", optimizer_name)
print("Steps:", steps)
print("Stepsize:", stepsize)
print("Mapping:", mapping)
print("Noise levels:", noise_levels)
print("Seeds:", seeds)
print("Ansatzes:", ansatzes)

## 2️⃣ Helper: simple robustness score

In [3]:
def auc_trapz(x, y):
    x = np.asarray(x, dtype=float)
    y = np.asarray(y, dtype=float)
    return float(np.trapz(y, x))


def summarise_noise_stats(result_dict):
    """
    Build a compact summary table from the output of
    run_vqe_ansatz_comparison(mode="noise_stats", ...).
    """
    x = np.asarray(result_dict["noise_levels"], dtype=float)
    rows = []
    for ans_name, data in result_dict["ansatzes"].items():
        deltaE_mean = np.asarray(data["deltaE_mean"], dtype=float)
        fid_mean = np.asarray(data["fidelity_mean"], dtype=float)

        rows.append(
            {
                "ansatz": ans_name,
                "AUC_deltaE": auc_trapz(x, np.abs(deltaE_mean)),
                "AUC_1_minus_fidelity": auc_trapz(x, 1.0 - fid_mean),
                "deltaE_at_max_noise": float(deltaE_mean[-1]),
                "fidelity_at_max_noise": float(fid_mean[-1]),
            }
        )

    rows = sorted(rows, key=lambda r: (r["AUC_deltaE"], r["AUC_1_minus_fidelity"]))
    return rows


def print_summary(rows, title):
    print("\n" + title)
    print("-" * len(title))
    header = (
        f"{'ansatz':<28s} "
        f"{'AUC|ΔE|':>12s} "
        f"{'AUC(1-F)':>12s} "
        f"{'ΔE@max':>12s} "
        f"{'F@max':>10s}"
    )
    print(header)
    print("-" * len(header))
    for r in rows:
        print(
            f"{r['ansatz']:<28s} "
            f"{r['AUC_deltaE']:>12.6f} "
            f"{r['AUC_1_minus_fidelity']:>12.6f} "
            f"{r['deltaE_at_max_noise']:>12.6f} "
            f"{r['fidelity_at_max_noise']:>10.4f}"
        )

## 3️⃣ Depolarizing noise: ansatz comparison (multi-seed)

In [ ]:
dep_results = run_vqe_ansatz_comparison(
    molecule=molecule,
    optimizer_name=optimizer_name,
    ansatzes=ansatzes,
    steps=steps,
    stepsize=stepsize,
    mode="noise_stats",
    noise_type="depolarizing",
    noise_levels=noise_levels,
    seeds=seeds,
    mapping=mapping,
    force=False,
    show=True,
    plot=True,
)

dep_summary = summarise_noise_stats(dep_results)
print_summary(dep_summary, title="Depolarizing noise — robustness summary")


## 4️⃣ Amplitude damping noise: ansatz comparison (multi-seed)

In [ ]:
amp_results = run_vqe_ansatz_comparison(
    molecule=molecule,
    optimizer_name=optimizer_name,
    ansatzes=ansatzes,
    steps=steps,
    stepsize=stepsize,
    mode="noise_stats",
    noise_type="amplitude",
    noise_levels=noise_levels,
    seeds=seeds,
    mapping=mapping,
    force=False,
    show=True,
    plot=True,
)

amp_summary = summarise_noise_stats(amp_results)
print_summary(amp_summary, title="Amplitude damping — robustness summary")